# COS chambers (Jonathan D. Müller & Rafat Qubaja)

Loads the raw data of Rafat Qubaja's branch chambers, measured in Yatir, applies corrections and calculates fluxes

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from scipy import stats

In [ ]:
# Initialization parameters

# Data input
project_path = '../'

project_path_amb   = project_path + '02_preprocessed_data/ambient/'
project_path_laser = project_path + '02_preprocessed_data/laser computer/'
project_path_par   = project_path + '02_preprocessed_data/PAR/'
project_path_tc    = project_path + '02_preprocessed_data/thermocouples/'
project_path_flow  = project_path + '02_preprocessed_data/flow/'

# Additional data, logbook etc.
project_path_additional = project_path + '00_additional_data/'
leaf_area_fn      = 'COS_leaf_area.xlsx'
chamber_status_fn = 'COS_chambers_logbook.xlsx'
cos_timing_fn     = 'COS_timing.xlsx'
par_logbook_fn    = 'PAR_sensors_logbook.xlsx'

# Output
project_path_output = project_path + '03_processed_data/'
output_fn = 'laser'

# List of months to process
# - If empty, all available data is processed, e.g. [  ]
# - Otherwise, specify using a string of year-month, e.g. ['2018-06'] or ['2017-03','2018-06']
month_list = [ '2020-09', '2021-03' ]


# Main parameters
par_threshold    = 20             # Threshold by which to determine whether to keep PAR
averaging_period = 30     # How many [s] of each measurement are used to average the data
keep_all_raw     = False  # If true, keep all data when creating the raw output. Otherwise, keep the data of the averaging period
#dew_threshold    = 98     # Percentage of RH when dew starts to form

# Choose how to find the ambient and open chambers value for each measurement. There are 2 possibilities:
# - interpolated: This interpolates ambient and OC measurements and adds them
# - next:         This simply finds the next ambient or OC, and adds it
#find_ambient     = 'next'
find_ambient     = 'interpolated'

# Leaf area parameters
proj_la_factor = np.pi # Factor used to convert total leaf area to projected leaf area
needle_diameter_irr = 0.82 # diameter [mm] of needles in the irrigation plot
needle_diameter_ctr = 0.89 # diameter [mm] of needles in the control plot
needle_diameter_LA = 0.85  # reference "wrong" diameter [mm] of needles used in the input leaf area data

# Pump parameters: Values different from these will be deleted
min_abs_flow = 3       # Minimum allowed pump flow
#max_delta_flow = 4     # Maximum difference between ambient and sample pumps
#max_stddev_flow = 0.5  # Maximum standard deviation within pump flows

## Load additionally needed data
- Leaf area logbook
- Chambers status (working or not)

In [ ]:
# Read additional data
#----------------------

# Read leaf area (la)
la = pd.read_excel(project_path_additional + leaf_area_fn, sheet_name='summary')
# Convert dates
la['starting_date'] = pd.to_datetime( la.starting_date, format='%Y-%m-%d', utc=True, errors='raise')
#display(la)

# Read chamber status (cs)
cs = pd.read_excel(project_path_additional + chamber_status_fn, usecols=list(range(0,5)), sheet_name='flow')
cs.dropna(subset=['year'], inplace=True)
# Convert dates
cs['timestamp'] = pd.to_datetime( cs.year.map(str) + "-" + cs.month.map(str) + "-" + cs.day.map(str), format='%Y.0-%m.0-%d.0', utc=True, errors='raise')
# Filter out errors (too many days in a month)
cs = cs[cs.timestamp.notnull()]
#display(cs)

# Read laser timing (lt) information, used to extract data when the chamber was closed
lt = pd.read_excel(project_path_additional + cos_timing_fn, sheet_name='Laser')
lt.rename(columns={'date':'timestamp'}, inplace=True)
lt['timestamp'] = pd.to_datetime( lt.timestamp, format='%Y-%m-%d', utc=True, errors='raise')
#display(lt)

# Read PAR sensor installation and information
par_log = pd.read_excel(project_path_additional + par_logbook_fn)
par_log.dropna(subset=['date'], inplace=True)
par_log.rename(columns={'cham no.':'chamber'}, inplace=True)
par_log.rename(columns={'MUX port ':'port'}, inplace=True)

In [ ]:
display(la)

# Functions

In [ ]:
# Physics: Unit converstions, fluxes, etc.
#-----------------------------------------

# Calculate the dewpoint temperature in C
def calculate_dewpointC(T_C, RH):
    dp = 243.04*(np.log(RH/100)+((17.625*T_C)/(243.04+T_C)))/(17.625-log(RH/100)-((17.625*T_C)/(243.04+T_C)))
    return(dp)

# Calculate saturation vapour pressure from pressure and temperature
# - 2 methods are available. Jones uses air pressure, Campbell & Norman do not
def calculate_es(T_C, P_Pa):
    # Jones p.348 (appendix 4)
    #es = (1.00072+(10**(-7)*P_Pa*(0.032+5.9*10**(-6)*T_C**2))) * (611.21*np.exp( (18.678-(T_C/234.5))*T_C/(257.14+T_C) ))

    # Eddypro manual: https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
    # Campbell & Norman (1998)
    T_K = T_C + 273.15
    es = T_K**(-8.2) * np.exp(77.345 + 0.0057*T_K - 7235 * T_K**(-1))
    return(es)

# Calculates VPD from different environmental variables
def calculate_VPD(T_C, h2o_mmol_mol, P_Pa):
    # Convert temperature from C to K
    T_K = T_C + 273.15
    
    h2o_mol_mol = h2o_mmol_mol * 10**(-3)

    # From Eddypro manual: https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
    R  = 8.314463                  # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897                # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802                # molecular weights of water vapour (kg mol-1)
    R_h2o = R / M_h2o              # Water vapor gas constant (J K-1 mol-1)
    es = calculate_es(T_C, P_Pa)   # Water vapor partial pressure at saturation (Pa)
    P_d = P_Pa - es                # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K)  # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d              # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa           # Air molar volume (vd, m3mol-1) 
    rho_h2o = h2o_mol_mol * M_h2o / v_a # Water vapor mole fraction
    e  = rho_h2o * R_h2o * T_K     # Water vapor partial pressure (Pa) 
    VPD = es - e                   # VPD (Pa)
    return(VPD)

# Converts water concentration [mmol_mol] to RH [%]
def convert_mmol_RH(T_C, h2o_mmol_mol, P_Pa):
    
    T_K = T_C + 273.15
    #es = calculate_es(T_C, P_Pa)
    #RH <- 0.263*P_Pa*((h2o_mmol_mol*18.02/28.97)/1000)*np.exp(17.67*(T_C)/(T_K-29.65))**(-1)
    #RH = 100 if (RH > 100) else RH
    #RH = np.nan if (RH < 5) else RH

    # From Eddypro manual: https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
    R  = 8.314463                  # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897                # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802                # molecular weights of water vapour (kg mol-1)
    R_h2o = R / M_h2o              # Water vapor gas constant (J K-1 mol-1)
    es = calculate_es(T_C, P_Pa)   # Water vapor partial pressure at saturation (Pa)
    P_d = P_Pa - es                # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K)  # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d              # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa           # Air molar volume (vd, m3mol-1) 
    rho_h2o = h2o_mmol_mol/1000 * M_h2o / v_a # Water vapor mole fraction
    e  = rho_h2o * R_h2o * T_K     # Water vapor partial pressure (Pa) 
    RH = e/es * 100                # RH (%)
    return(RH)

# Density of dry air
# - https://en.wikipedia.org/wiki/Density_of_air
def calculate_rho_dry_air(T_C, P_Pa):
    # Constants
    R_dry_air = 287.058     # [J/(kg·K)] Specific gas const dry air
    # Calculations
    T_K = T_C + 273.15
    rho_dry_air = P_Pa / (R_dry_air * T_K) # Density of dry air (use for approximation)
    return(rho_dry_air)

# Density of moist air
def calculate_rho_moist_air(T_C, h2o_mmol_mol, P_Pa):
    # Temperature in K
    T_K = T_C + 273.15

    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    es = calculate_es(T_C, P_Pa) # Saturation vapour pressure (Pa)
    P_d = P_Pa - es              # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1) 
    rho_h2o = h2o_mmol_mol/1000 * M_h2o / v_a # Water vapor mole fraction

    # Moist air mass density (ρa, kg m-3) 
    rho_air = rho_d + rho_h2o

    return(rho_air)

# Dry air heat capacity at constant pressure
# cp_d in [J kg-1 K-1]
 # https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
def calculate_cp_dry_air(T_C):
    cp = 1005 + ((T_C + 23.12)**2)/3364
    return(cp)

# Specific heat capacity of moist air at constant pressure
# cp_m in [J kg-1 K-1]
# https://www.licor.com/env/support/EddyPro/topics/calculate-micromet-variables.html
def calculate_cp_moist_air(T_C, h2o_mmol_mol, P_Pa):
    # Temperature in K
    T_K = T_C + 273.15

    # RH
    RH = convert_mmol_RH(T_C, h2o_mmol_mol, P_Pa)

    # Water vapor heat capacity at constant pressure (cp_h2o, J kg-1 K-1)
    cp_h2o = 1859 + 0.13*RH + (0.193 + 5.6*10**(-3) * RH)*T_C + (10**(-3) + 5 * 10**(-5)*RH)*T_C**2

    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    es = calculate_es(T_C, P_Pa) # Saturation vapour pressure (Pa)
    P_d = P_Pa - es              # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1) 
    rho_h2o = h2o_mmol_mol/1000 * M_h2o / v_a # Water vapor mole fraction

    # Moist air mass density (ρa, kg m-3) 
    rho_air = rho_d + rho_h2o

    # Specific humidity (Q, kg kg-1) 
    Q = rho_h2o / rho_air

    # cp_moist
    cp = calculate_cp_dry_air(T_C) * (1-Q) + cp_h2o * Q
    return(cp)

# Calculates the flux of water
def calculate_h2o_flux(T_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, airflow_lpm, area_m2):
    # Temperature in K
    T_K = T_C + 273.15
    
    # Unit conversions
    h2o_mol_mol_ambient = h2o_mmol_mol_ambient * 10**(-3)
    h2o_mol_mol_chamber = h2o_mmol_mol_chamber * 10**(-3)
    
    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    es = calculate_es(T_C, P_Pa) # Saturation vapour pressure (Pa)
    P_d = P_Pa - es              # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (v_d, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (v_a, m3mol-1)
    
    # Convert airflow from LPM to mol.s-1
    airflow_m3_s = airflow_lpm / 1000 / 60 # airflow conversion to total flow [m3.s-1]
    airflow_mol_s = airflow_m3_s / v_a     # airflow conversion to moist air [mol.s-1]
    
    # Note: We are measuring the outflow from the chamber, not the inflow (as opposed to the LI-6400)
    #       Therefore, we need to change eq. 1-2 (p. 1-7) to be ue = uo - sE
    #       This leads to eq. 1-3 changing to sE = uo*wo - (uo - sE)*we
    #       Finally, eq. 1-4 becomes E = uo*(wo - we) / (s*(1 - we))
    #       Where uo = airflow_mol_s
    #             wo = h2o_mol_mol_chamber (outgoing mol fraction)
    #             we = h2o_mol_mol_ambient (entering mol fraction)
    #             s  = area_m2 (leaf area)
      
    # LI-6400 manual, eq. 1-7
    h2o_flux = (airflow_mol_s * (h2o_mol_mol_chamber - h2o_mol_mol_ambient)) / (area_m2 * (1 - h2o_mol_mol_ambient))
    
    return(h2o_flux) # mol.m-2.s-1

# Inspired from LI-6400 manual: Uses the water flux because water changes the air density. If stomata are open and H2o is added, the gas is more 'diluted'
def calculate_gas_flux(T_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, gas_mol_mol_ambient, gas_mol_mol_chamber, airflow_lpm, area_m2):
    # Temperature in K
    T_K = T_C + 273.15
    
    # Unit conversions
    h2o_mol_mol_ambient = h2o_mmol_mol_ambient * 10**(-3)
    h2o_mol_mol_chamber = h2o_mmol_mol_chamber * 10**(-3)
    
    # Preparations
    R     = 8.314463             # Ideal gas constant (J K-1 mol-1)
    M_d   = 0.02897              # molecular weights of dry air (kg mol-1)
    M_h2o = 0.01802              # molecular weights of water vapour (kg mol-1)
    es = calculate_es(T_C, P_Pa) # Saturation vapour pressure (Pa)
    P_d = P_Pa - es              # Dry air partial pressure (P_d, P_a)
    rho_d = P_d / (R / M_d * T_K) # Dry air mass density (rho_d, kg m-3)
    v_d = M_d / rho_d            # Dry air molar volume (vd, m3 mol-1)
    v_a = v_d * P_d/P_Pa         # Air molar volume (vd, m3mol-1)
    
    # Convert airflow from LPM to mol.s-1
    airflow_m3_s = airflow_lpm / 1000 / 60 # airflow conversion to total flow [m3.s-1]
    airflow_mol_s = airflow_m3_s / v_a     # airflow conversion to moist air [mol.s-1]
    
    # H2O transpiration flux [mol.m-2.s-1]
    h2o_flux_mol_m2_s = calculate_h2o_flux(T_C, P_Pa, h2o_mmol_mol_ambient, h2o_mmol_mol_chamber, airflow_lpm, area_m2)
    
    # Note: We are measuring the outflow from the chamber, not the inflow (as opposed to the LI-6400)
    #       Therefore, we need to change eq. 1-2 (p. 1-7) to be ue = uo - sE
    #       The LI-6400 uses assimilation, which is a positive flux. Previously, we used negative numbers due to CO2 uptake
    #       The sign should reflect the direction of the flux, i.e. emission is positive, and uptake negative, so assimilation should be negative
    #       The following calculations will therefore change eq. 1-11 (p. 1-9) to sa = uo*co - ue*ce
    #       This leads to eq. 1-12 (p. 1-9) changing to sa = uo*co - (uo - sE)*ce
    #       Finally, eq. 1-13 becomes a = uo*(co-ce) / s + E*ce
    #       Where uo = airflow_mol_s
    #             co = co2_mol_mol_chamber (outgoing mol fraction)
    #             ce = co2_mol_mol_ambient (entering mol fraction)
    #             s  = area_m2 (leaf area)
    #             E  = water flux (calculated above)
    
    # LI-6400 manual, eq. 1-13 (p. 1-10)
    gas_flux = (airflow_mol_s * (gas_mol_mol_chamber - gas_mol_mol_ambient)) / area_m2 + h2o_flux_mol_m2_s * gas_mol_mol_ambient
    
    return(gas_flux) # mol.m-2.s-1

In [ ]:
# Supporting functions
#-----------------------

# Assign block IDs for grouping and averaging in calculations
def create_block_id(temp):
    temp = temp.copy()
    blockID = 0
    blockIDs = []
    # Create an ID variable, used only here
    temp['id_var'] = temp['chamber'].astype('str') + temp['status']
    x_old = temp["id_var"].tolist()[0]

    for x in temp["id_var"].tolist():
        # If same chamber, give same blockID
        if x == x_old:
            blockIDs.append(blockID)
            continue
        # If switch to new chamber, give new blockID
        else:
            blockID += 1
            blockIDs.append(blockID)
        x_old = x
    
    # Drop ID variable
    temp.drop('id_var', axis=1, inplace=True)
    
    # Add block ID
    temp.loc[:,'chamber_block'] = blockIDs
    
    return(temp)

# Calculate the standard deviation of means of means
# - provide a list of standard deviation columns of each initial mean
# - returns a list of standard deviations, by row
def std_means_of_means(sds):
        # Calculate a list of variances
        var = []
        for i in sds:
            var.append(i**2)
        # Calculate the mean
        var_mean = np.mean(var, axis=0)
        # Calculate the standard deviation
        sd_mean = np.sqrt(var_mean)
        return(sd_mean)

In [ ]:
# Main functions
#---------------

# Delete all relevant files in a folder.
# - Used to remove 1min monthly files before running to prevent appending to existing files
def empty_dir(directory):
    files = glob.glob(directory + '*')
    for f in files:
        month_id = f[-10:-6] + '-' + f[-6:-4]
        if(month_id in month_list):
            print('Remove ' + f)
            os.remove(f)
        if(not month_list):
            print('Remove ' + f)
            os.remove(f)
    pass

# Read an input file
def read_data_file(input_fn):
    df = pd.read_csv(input_fn, na_values=["NAN"])
    df['timestamp'] = pd.to_datetime( df.timestamp, format='%Y-%m-%d %H:%M:%S', utc=True, errors="raise")
    df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
    return(df)

# This finds the file corresponding to a specific date and loads it
def find_and_load_file(fn_list, date_idx):
    data_fn = [i for i in fn_list if date_idx in i][0] if len([i for i in fn_list if date_idx in i]) > 0 else None
    if(data_fn is None):
        temp_df = read_data_file(fn_list[0])
        temp_df = temp_df.iloc[0:0]
    else:
        temp_df  = read_data_file(data_fn)
    return(temp_df)

# Make means of all gases, because there are multiple measurements. Also add standard deviations
def make_gas_means(temp):
    temp = temp.copy()
    # Convert units
    temp['OCS.1'] = temp['OCS.1'] * 1000
    temp['OCS.2'] = temp['OCS.2'] * 1000
    temp['CO2.1'] = temp['CO2.1'] / 1000
    temp['CO2.2'] = temp['CO2.2'] / 1000
    temp['CO2.3'] = temp['CO2.3'] / 1000
    temp['CO2.4'] = temp['CO2.4'] / 1000
    temp['H2O.1'] = temp['H2O.1'] / 10**6
    temp['CO.1'] = temp['CO.1'] * 1
    # drop bad columns
    temp.drop('CO2.3', axis=1, inplace=True)
    temp.drop('CO2.4', axis=1, inplace=True)
    # Rename columns
    temp.rename(columns={'OCS.1':'cos.1.pmol_mol'}, inplace=True)
    temp.rename(columns={'OCS.2':'cos.2.pmol_mol'}, inplace=True)
    temp.rename(columns={'CO2.1':'co2.1.umol_mol'}, inplace=True)
    temp.rename(columns={'CO2.2':'co2.2.umol_mol'}, inplace=True)
    temp.rename(columns={'H2O.1':'h2o.mmol_mol'}, inplace=True)
    temp.rename(columns={'CO.1':'co.nmol_mol'}, inplace=True)
    # Make mean of data
    temp['cos.mean.pmol_mol'] = temp[['cos.1.pmol_mol','cos.2.pmol_mol']].mean(axis=1)
    temp['co2.mean.umol_mol'] = temp[['co2.1.umol_mol','co2.2.umol_mol']].mean(axis=1)
    # Add standard deviation of data
    temp['cos.mean_sd.pmol_mol'] = temp[['cos.1.pmol_mol','cos.2.pmol_mol']].std(axis=1)
    temp['co2.mean_sd.umol_mol'] = temp[['co2.1.umol_mol','co2.2.umol_mol']].std(axis=1)
    return(temp)

# Read the PAR log, and extract the relevant data 
def get_PAR(par_df, this_day):
    raw_par_df = par_df.copy()
    
    # Create empty PAR df
    par_df = raw_par_df[['timestamp']].copy()
    for i in range(1,17):
        par_df['par.ch' + str(i) + '.umol_m2_s'] = np.nan
    
    # Find relevant log entries
    previous_log_date = par_log['date'].unique()[0]
    log_entry_date = 0
    for i in par_log['date'].unique():
        current_date = pd.to_datetime(str(this_day), format='%Y%m%d')
        # Find the entry that's in between the current date and the previous
        if((current_date > previous_log_date) and (current_date < i)):
            log_entry_date = previous_log_date
        # If the current date is larger than the last entry, keep the last entry
        if(current_date > par_log['date'].unique()[-1]):
            log_entry_date = i
        # If the entry is on a measurement day, make it 0 so that NAs are filled in
        if(current_date == i):
            log_entry_date = 0
        previous_log_date = i

    # Find the relevant chamber and port, and fill the df
    for i in par_log.loc[(par_log['date'] == log_entry_date) & (par_log['status'] == 'good'),'chamber']:
        port = int(par_log.loc[(par_log['date'] == log_entry_date) & (par_log['chamber'] == i),'port'].values[0])
        par_df['par.ch' + str(int(i)) + '.umol_m2_s'] = raw_par_df['par_Avg(' + str(port) + ')']
    
    # The sensor values need to be multiplied by 100 to convert mV to umol.m-2.s-1
    for i in range(1,17):
        par_df['par.ch' + str(i) + '.umol_m2_s'] = par_df['par.ch' + str(i) + '.umol_m2_s'] * 100
        
    # Keep only chambers 4 & 16
    for i in [1,2,3,5,6,7,8,9,10,11,12,13,14,15]:
        par_df.drop('par.ch' + str(i) + '.umol_m2_s', axis=1, inplace=True)
        
    return(par_df)

def add_ambient(temp, amb_df):
    temp = temp.copy()
    
    # Drop unnecessary columns
    amb_df.drop('dayid', axis=1, inplace=True)
    
    # Merge PAR data
    merged = temp.merge(amb_df, on="timestamp", how="left")
    # Interpolate
    merged['temp.air.ambient.c'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    merged['par.ambient.umol_m2_s'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    merged['h2o.irga.ambient.mmol_mol'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    merged['co2.irga.ambient.umol_mol'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    
    return(merged)

def add_PAR(temp, par_df):
    temp = temp.copy()
        
    # Merge PAR data
    merged = temp.merge(par_df, on="timestamp", how="left")
    # Interpolate
    for i in [4,16]:
        merged['par.ch' + str(i) + '.umol_m2_s'].interpolate(method='linear', limit=120, limit_direction='both', limit_area='inside', inplace=True)
        
    # Night-time corrections, by cycling through chambers: Set to 0 during the night
    merged.loc[merged['par.ambient.umol_m2_s'] < (par_threshold/2), 'par.ambient.umol_m2_s'] = 0
    merged.loc[(merged['timestamp'].dt.hour < 5) | (merged['timestamp'].dt.hour > 21) , 'par.ambient.umol_m2_s'] = 0 # Define the night by the maximum of June 21
    for i in [4,16]:
        merged.loc[(merged['par.ch' + str(i) + '.umol_m2_s'] < par_threshold), 'par.ch' + str(i) + '.umol_m2_s'] = 0 # All PAR values should be 0 at night. Small values are errors
        merged.loc[(merged['par.ambient.umol_m2_s'] == 0) & (merged['par.ch' + str(i) + '.umol_m2_s'] > par_threshold), 'par.ch' + str(i) + '.umol_m2_s'] = 0 # Remove large night-time values
        merged.loc[(merged['par.ambient.umol_m2_s'] == 0) & (merged['par.ch' + str(i) + '.umol_m2_s'] > 0), 'par.ch' + str(i) + '.umol_m2_s'] = 0                  # Apogee non-zero at night
        
    return(merged)

def add_temperature(temp, tc_df):
    temp = temp.copy()
    
    # Drop unnecessary columns
    tc_df.drop('dayid', axis=1, inplace=True)
    
    # Rename columns
    tc_df.rename(columns={'TcRafat(1)':'temp.leaf.ch4.c'}, inplace=True)
    tc_df.rename(columns={'TcRafat(2)':'temp.air.ch4.c'}, inplace=True)
    tc_df.rename(columns={'TcRafat(3)':'temp.leaf.ch16.c'}, inplace=True)
    tc_df.rename(columns={'TcRafat(4)':'temp.air.ch16.c'}, inplace=True)
    tc_df.rename(columns={'TcRafat(6)':'temp.laserbox.c'}, inplace=True)
    # When they made new names
    tc_df.rename(columns={'TcRafat_Avg(1)':'temp.leaf.ch4.c'}, inplace=True)
    tc_df.rename(columns={'TcRafat_Avg(2)':'temp.air.ch4.c'}, inplace=True)
    tc_df.rename(columns={'TcRafat_Avg(3)':'temp.leaf.ch16.c'}, inplace=True)
    tc_df.rename(columns={'TcRafat_Avg(4)':'temp.air.ch16.c'}, inplace=True)
    tc_df.rename(columns={'TcRafat_Avg(6)':'temp.laserbox.c'}, inplace=True)
    
    # select columns and rename
    tc_df_selected = tc_df[['timestamp','temp.leaf.ch4.c','temp.air.ch4.c','temp.leaf.ch16.c','temp.air.ch16.c','temp.laserbox.c']]
    
    # Merge PAR data
    merged = temp.merge(tc_df_selected, on="timestamp", how="left")
    
    # Interpolate
    merged['temp.leaf.ch4.c'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    merged['temp.air.ch4.c'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    merged['temp.leaf.ch16.c'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    merged['temp.air.ch16.c'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    merged['temp.laserbox.c'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)

    return(merged)

def add_pump_flow(temp, flow_df):
    temp = temp.copy()
    
    # Rename columns
    flow_df.rename(columns={'FlowMeter(1)':'pump.flow.ch4.slpm'}, inplace=True)
    flow_df.rename(columns={'FlowMeter(2)':'pump.flow.ch16.slpm'}, inplace=True)
    
    # select columns and rename
    flow_df_selected = flow_df[['timestamp', 'pump.flow.ch4.slpm', 'pump.flow.ch16.slpm']]
    
    # Merge flow data
    merged = temp.merge(flow_df_selected, on="timestamp", how="left")
    
    # Interpolate
    merged['pump.flow.ch4.slpm'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    merged['pump.flow.ch16.slpm'].interpolate(method='linear', limit=4, limit_direction='both', limit_area='inside', inplace=True)
    
    return(merged)
    
def make_counter_and_chamber(temp, this_day):
    temp = temp.copy()
    
    # Extract the correct timing from the log file. If it's before the first available date, drop all data
    if(lt['timestamp'].unique()[0] >= pd.to_datetime(str(this_day), format='%Y%m%d', utc=True)): # Data from before the log file
        temp['status'] = np.nan
        temp['chamber'] = np.nan
        temp['counter'] = np.nan
        temp['chamber_block'] = np.nan
        temp['plot'] = np.nan
        temp.drop(temp.index, inplace=True)
        return(temp)
    # Extract the correct timing from the log file
    relevant_date = lt.loc[lt['timestamp'] < this_day,'timestamp'].unique()[-1]
    timing_dt = lt.loc[lt['timestamp'] == relevant_date]
    
    # Mark the chambers, and add chamber 0 for data that needs to be removed
    temp['status'] = ''
    temp['chamber'] = 0 # remove if it remains 0
    for index, row in timing_dt.iterrows():
        temp.loc[(temp['timestamp'].dt.minute >= row['start_min']) & (temp['timestamp'].dt.minute <= row['end_min']), 'status'] = row['type']
        if(row['plot'] == 'irr'):
            temp.loc[(temp['timestamp'].dt.minute >= row['start_min']) & (temp['timestamp'].dt.minute <= row['end_min']), 'chamber'] = 4
        if(row['plot'] == 'con'):
            temp.loc[(temp['timestamp'].dt.minute >= row['start_min']) & (temp['timestamp'].dt.minute <= row['end_min']), 'chamber'] = 16
    # change the chamber assignement for times before the 28th of august 2020, when measurements were every 2h
    temp.loc[(temp['timestamp'] < '2020-08-28') & ((temp['timestamp'].dt.hour % 2) == 0) & (temp['chamber'] == 16), 'chamber'] = 4
    
    # Add counter
    temp = create_block_id(temp)
    temp['counter'] = temp.groupby(['chamber_block']).cumcount()
    # Revert the counter to end at 0
    for name, group in temp.groupby(['chamber_block']):
        temp.loc[temp['chamber_block'] == name, 'counter'] = group['counter'] - group['timestamp'].count() + 1
        
    # Convert to integer
    temp['chamber'] = temp['chamber'].astype(int)
    # Remove chamber number when not relevant
    temp.loc[~temp['status'].isin(['oc','cc','']), 'chamber'] = np.nan
    # Add plot information
    # - Chamber 4 is irrigation
    # - Chamber 16 is control
    temp['plot'] = np.nan
    temp.loc[temp['chamber'] == 4, 'plot'] = 'irrigation'
    temp.loc[temp['chamber'] == 16, 'plot'] = 'control'
    temp.loc[temp['status'] == 'amb', 'plot'] = 'ambient'
    
    # Chamber 0 is a dummy that needs to be removed
    temp = temp.loc[temp['chamber'] != 0]
    
    return(temp)

def add_current_variable(temp):
    temp = temp.copy()
    # Add a 'current' column for par temperature and flowrate, which has the reading for the current chamber
    temp['par.current.chamber.umol_m2_s'] = np.nan
    temp['temp.leaf.current.chamber.c'] = np.nan
    temp['temp.air.current.chamber.c'] = np.nan
    temp['pump.flow.current.chamber.slpm'] = np.nan
    # Loop through chambers and fill in the gaps
    for i in np.unique(temp['chamber'].dropna()):
        temp.loc[temp['chamber'] == i, 'par.current.chamber.umol_m2_s'] = temp.loc[temp['chamber'] == i, 'par.ch' + str(int(i)) + '.umol_m2_s']
        temp.loc[temp['chamber'] == i, 'temp.leaf.current.chamber.c'] = temp.loc[temp['chamber'] == i, 'temp.leaf.ch' + str(int(i)) + '.c']
        temp.loc[temp['chamber'] == i, 'temp.air.current.chamber.c'] = temp.loc[temp['chamber'] == i, 'temp.air.ch' + str(int(i)) + '.c']
        temp.loc[temp['chamber'] == i, 'pump.flow.current.chamber.slpm'] = temp.loc[temp['chamber'] == i, 'pump.flow.ch' + str(int(i)) + '.slpm']
    temp.loc[temp['chamber'].isna(), 'par.current.chamber.umol_m2_s'] = temp.loc[temp['chamber'].isna(), 'par.ambient.umol_m2_s']
    temp.loc[temp['chamber'].isna(), 'temp.leaf.current.chamber.c'] = np.nan
    temp.loc[temp['chamber'].isna(), 'temp.air.current.chamber.c'] = temp.loc[temp['chamber'].isna(), 'temp.air.ambient.c']
    temp.loc[temp['chamber'].isna(), 'pump.flow.current.chamber.slpm'] = np.nan
    return(temp)

def add_leaf_area(temp, this_day):
    temp = temp.copy()
    
    # Extract the correct timing from the log file
    relevant_date = la.loc[la['starting_date'] <= this_day,'starting_date'].unique()[-1]
    la_df = la.loc[la['starting_date'] == relevant_date]
    
    # Add leaf area
    temp['leaf.area.m2'] = np.nan
    for index, row in la_df.iterrows():
        temp.loc[(temp['chamber'] == row['chamber']), 'leaf.area.m2'] = row['la']
    
    # Leaf area was calculated for a diameter of 0.85mm, but we need to fix it for the real diameter of each plot
    temp.loc[temp['chamber'] == 4, 'leaf.area.m2'] = temp.loc[temp['chamber'] == 4, 'leaf.area.m2'] * (needle_diameter_irr/needle_diameter_LA)
    temp.loc[temp['chamber'] == 16, 'leaf.area.m2'] = temp.loc[temp['chamber'] == 16, 'leaf.area.m2'] * (needle_diameter_ctr/needle_diameter_LA)
    
    return(temp)

# Remove data from the bad chambers, based on the logbook
def add_log_flow(temp):
    temp = temp.copy()
    
    # Identify day
    day = temp['timestamp'].dt.normalize().unique()[0]
    
    # Keep only relevant days, make a list of bad chambers
    cs_df = cs.loc[cs['timestamp'] == day][cs.columns[3:5]]
    
    # Replace the pump flow if a constant was given in the logbook, or remove values if the pump wasn't working (according to the logbook)
    for chamber in cs_df.columns.tolist():
        if(cs_df[chamber].values[0] > 0):
            temp.loc[temp['chamber'] == chamber, 'pump.flow.current.chamber.slpm'] = cs_df[chamber].values[0]
        if(cs_df[chamber].values[0] == 0): # pump not working
            temp.loc[temp['chamber'] == chamber, 'pump.flow.current.chamber.slpm'] = np.nan
    
    # Consider the pump as not working at a flow below the limit
    temp.loc[temp['pump.flow.current.chamber.slpm'] <= min_abs_flow, 'pump.flow.current.chamber.slpm'] = np.nan
    
    return(temp)

# Remove data from the bad chambers, based on the logbook
def remove_bad_chambers(temp):
    temp = temp.copy()
    
    # Identify day
    day = temp['timestamp'].dt.normalize().unique()[0]
    
    # Keep only relevant days, make a list of bad chambers
    cs_df = cs.loc[cs['timestamp'] == day][cs.columns[3:5]]
    bad_chambers = cs_df.columns[(cs_df == 0).all()].tolist()
    
    # Show information
    print("    Data bad in chambers: ", bad_chambers)
    
    # Remove data when entire chamber was bad
    # COS (Carbonyl sulfide)
    temp.loc[temp['chamber'].isin(bad_chambers),'cos.1.pmol_mol'] = np.nan
    temp.loc[temp['chamber'].isin(bad_chambers),'cos.2.pmol_mol'] = np.nan
    temp.loc[temp['chamber'].isin(bad_chambers),'cos.mean.pmol_mol'] = np.nan
    temp.loc[temp['chamber'].isin(bad_chambers),'cos.mean_sd.pmol_mol'] = np.nan
    # CO2 (Carbon dioxide)
    temp.loc[temp['chamber'].isin(bad_chambers),'co2.1.umol_mol'] = np.nan
    temp.loc[temp['chamber'].isin(bad_chambers),'co2.2.umol_mol'] = np.nan
    temp.loc[temp['chamber'].isin(bad_chambers),'co2.mean.umol_mol'] = np.nan
    temp.loc[temp['chamber'].isin(bad_chambers),'co2.mean_sd.umol_mol'] = np.nan
    # H2O (Water)
    temp.loc[temp['chamber'].isin(bad_chambers),'h2o.mmol_mol'] = np.nan
    # CO (Carbon monoxide)
    temp.loc[temp['chamber'].isin(bad_chambers),'co.nmol_mol'] = np.nan
    
    return(temp)

# Writes output files in full temporal resolution
def write_output_file(out_df, date_idx, out_dir):
    # Check if output folders exist. If not, create
    month_dir = str(date_idx)[0:4] + "-" + str(date_idx)[4:6]
    if(not os.path.exists(out_dir + month_dir)): # make directory if it doesn't exist
        os.makedirs(out_dir + month_dir)
    # Create file name
    out_fn = out_dir + month_dir + "/" + output_fn + "_" + str(date_idx) + ".csv"
    print(str(len(out_df.dayid)), out_fn) # Shows final file size)
    # organise data for output
    temp_df = out_df
    # Before saving, remove the index
    temp_df.drop('dayid', axis=1, inplace=True)
    #temp_df.drop('chamber_block', axis=1, inplace=True)
    # Move timestamp column to the front
    col = temp_df.pop('counter')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('chamber_block')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('status')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('plot')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('chamber')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    col = temp_df.pop('timestamp')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    # Remove timezone information
    temp_df['timestamp'] = temp_df['timestamp'].dt.tz_localize(None)
    # Write data
    temp_df.to_csv(out_fn, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S') # Save file
    pass

# Removes all data that is not in the relevant timeframe
def remove_obsolete_data(temp, seconds):
    temp = temp.copy()
    # Remove data, if 'keep_all_raw' is 'True', keep all data
    if(keep_all_raw):
        pass
    else:
        temp = temp.loc[temp['counter'] > (-seconds)]
    return(temp)

# Calculate means and standard deviations of all data columns
def make_means(temp, seconds):
    temp = temp.copy()
    
    # Prepare timestamps: Convert to numbers so that averaging becomes possible
    temp['timestamp'] = temp['timestamp'].astype('int64')
    
    # Remove obsolete data if it wasn't removed previously
    if(keep_all_raw):
        temp = temp.loc[temp['counter'] > (-seconds)]
    
    # Keep relevant identification columns
    id_cols = temp[['chamber_block','plot','status']].drop_duplicates()
    
    # Make standard deviations
    sd_df = temp.groupby(['chamber_block']).std()
    sd_df.reset_index(inplace=True)
    sd_df.drop('counter', axis=1, inplace=True)
    sd_df.drop('timestamp', axis=1, inplace=True)
    sd_df.drop('chamber', axis=1, inplace=True)
    sd_df.drop('leaf.area.m2', axis=1, inplace=True)
    sd_df = sd_df.add_suffix('.sd')
    sd_df.rename(columns={'chamber_block.sd':'chamber_block'}, inplace=True)
    # Make means
    avg_df = temp.groupby(['chamber_block']).mean()
    avg_df.drop('counter', axis=1, inplace=True)
    avg_df.reset_index(inplace=True)
    # Convert averaged 'timestamps stored as numbers' back to (rounded) timestamps
    avg_df['timestamp'] = avg_df['timestamp'].astype('datetime64[ns]').dt.round('10s')
    
    # Merge means with standard deviations
    summarised = avg_df.merge(sd_df, on="chamber_block", how="left")
    # Remove weird indices
    summarised.reset_index(inplace=True)
    summarised.drop('index', axis=1, inplace=True)
    
    # Add identification columns (status and plot). We need to do this because text is not preserved when making averages
    summarised = summarised.merge(id_cols, on="chamber_block", how="left")
    # Move important columns to the front
    col = summarised.pop('status')
    summarised.insert(0, col.name, col, allow_duplicates=True)
    col = summarised.pop('plot')
    summarised.insert(0, col.name, col, allow_duplicates=True)
    col = summarised.pop('chamber')
    summarised.insert(0, col.name, col, allow_duplicates=True)
    col = summarised.pop('timestamp')
    summarised.insert(0, col.name, col, allow_duplicates=True)
    
    # Fix: Calculate mean of means, and their standard deviations, of the gases with multiple columns (cos, co2)
    # (1) Remove previous standard deviations
    summarised.drop('cos.mean_sd.pmol_mol', axis=1, inplace=True)
    summarised.drop('co2.mean_sd.umol_mol', axis=1, inplace=True)
    summarised.drop('cos.mean_sd.pmol_mol.sd', axis=1, inplace=True)
    summarised.drop('co2.mean_sd.umol_mol.sd', axis=1, inplace=True)
    # (2) Calculate mean of means
    summarised['cos.mean.pmol_mol'] = summarised[['cos.1.pmol_mol','cos.2.pmol_mol']].mean(axis=1)
    summarised['co2.mean.umol_mol'] = summarised[['co2.1.umol_mol','co2.2.umol_mol']].mean(axis=1)
    # (3) Calculate standard deviation of 'means of means'
    summarised['cos.mean.pmol_mol.sd'] = std_means_of_means([summarised['cos.1.pmol_mol.sd'], summarised['cos.2.pmol_mol.sd']])
    summarised['co2.mean.umol_mol.sd'] = std_means_of_means([summarised['co2.1.umol_mol.sd'], summarised['co2.2.umol_mol.sd']])
    
    return(summarised)

# Interpolates ambient and open chamber readings into 1min readings. Then these are taken for the ambient reading
def find_corresponding_ambient_interpolated(temp):
    temp = temp.copy()
    
    # Create empty dataframe with a 1min frequency
    idx = pd.date_range(start=temp["timestamp"].tolist()[0],
                        end=temp["timestamp"].tolist()[-1],
                        freq="min")
    time_df = pd.DataFrame(idx, index=None, columns=["timestamp"])
    
    # List of relevant columns
    col_list = ['timestamp','cos.1.pmol_mol','cos.2.pmol_mol','co2.1.umol_mol','co2.2.umol_mol','h2o.mmol_mol','co.nmol_mol','cos.mean.pmol_mol','co2.mean.umol_mol']
    # Extract ambient, open chamber (4 & 16)
    ambient_df = time_df.merge(temp.loc[(temp['status'] == 'amb'),col_list], on="timestamp", how="left")
    ch4_df     = time_df.merge(temp.loc[(temp['chamber'] == 4) & (temp['status'] == 'oc'),col_list], on="timestamp", how="left")
    ch16_df    = time_df.merge(temp.loc[(temp['chamber'] == 16) & (temp['status'] == 'oc'),col_list], on="timestamp", how="left")
    # Interpolate data
    for i in col_list[1:]:
        ambient_df[i] = ambient_df[i].interpolate(method='linear', limit=60, limit_direction='both')
        ch4_df[i]     = ch4_df[i].interpolate(method='linear', limit=60, limit_direction='both')
        ch16_df[i]    = ch16_df[i].interpolate(method='linear', limit=60, limit_direction='both')
    ch4_df['chamber'] = 4
    ch16_df['chamber'] = 16
    # Make one dataframe for open chambers
    oc_df = ch4_df.append(ch16_df)
    # Rename columns
    ambient_df = ambient_df.add_suffix('.ambient')
    ambient_df.rename(columns={'timestamp.ambient':'timestamp'}, inplace=True)
    oc_df = oc_df.add_suffix('.oc')
    oc_df.rename(columns={'timestamp.oc':'timestamp'}, inplace=True)
    oc_df.rename(columns={'chamber.oc':'chamber'}, inplace=True)
    # Merge with data
    merged = temp.loc[temp['chamber'].notna()].merge(ambient_df, on="timestamp", how="left")
    merged = merged.merge(oc_df, on=["timestamp","chamber"], how="left")
    
    return(merged)

# Find the next ambient or open chamber after the current measurement, mark it, then append it
def find_corresponding_ambient_next(temp):
    temp = temp.copy()
    
    # List of relevant columns
    col_list = ['chamber','status','chamber_block','cos.1.pmol_mol','cos.2.pmol_mol','co2.1.umol_mol','co2.2.umol_mol','h2o.mmol_mol','co.nmol_mol','cos.mean.pmol_mol','co2.mean.umol_mol']
    
    # Prepare the data: Create 
    ambient_df    = temp.loc[(temp['status'] == 'amb'),col_list]
    ambient_oc_df = temp.loc[(temp['status'] == 'amb'),col_list]
    oc_df         = temp.loc[(temp['status'] == 'oc'),col_list]
    
    # Mark the relevant chamber_blocks
    # - Used for closed chamber
    ch4_amb_found  = True
    ch16_amb_found = True
    current_ch4_block  = np.nan
    current_ch16_block = np.nan
    # - Used for open chamber
    ch4_oc_amb_found  = True
    ch16_oc_amb_found = True
    current_oc_ch4_block  = np.nan
    current_oc_ch16_block = np.nan
    # - prepare empty columns
    oc_df['corresponding_chamber_block']         = np.nan
    ambient_df['corresponding_chamber_block']    = np.nan
    ambient_oc_df['corresponding_chamber_block'] = np.nan
    # - Find and mark everything
    for index, row in temp.iterrows():
        # Find current closed chamber
        if((row['chamber'] == 4) & (row['status'] == 'cc')):
            current_ch4_block = row['chamber_block']
            ch4_amb_found = False
        if((row['chamber'] == 16) & (row['status'] == 'cc')):
            current_ch16_block = row['chamber_block']
            ch16_amb_found = False
        # Connect open ohamber with closed chamber
        oc_df.loc[(row['chamber_block'] == oc_df['chamber_block']) & (oc_df['chamber'] ==  4) & (current_ch4_block  >= row['chamber_block']-2),'corresponding_chamber_block'] = current_ch4_block
        oc_df.loc[(row['chamber_block'] == oc_df['chamber_block']) & (oc_df['chamber'] == 16) & (current_ch16_block >= row['chamber_block']-2),'corresponding_chamber_block'] = current_ch16_block
        # Connect ambient with closed chamber
        if((ch4_amb_found == False) & (row['status'] == 'amb')):
            ambient_df.loc[row['chamber_block'] == ambient_df['chamber_block'],'corresponding_chamber_block'] = current_ch4_block
            ch4_amb_found = True
        if((ch16_amb_found == False) & (row['status'] == 'amb')):
            ambient_df.loc[row['chamber_block'] == ambient_df['chamber_block'],'corresponding_chamber_block'] = current_ch16_block
            ch16_amb_found = True
        # Ambient for open chamber
        if((row['chamber'] == 4) & (row['status'] == 'oc')):
            current_oc_ch4_block = row['chamber_block']
            ch4_oc_amb_found = False
        if((row['chamber'] == 16) & (row['status'] == 'oc')):
            current_oc_ch16_block = row['chamber_block']
            ch16_oc_amb_found = False
        if((ch4_oc_amb_found == False) & (row['status'] == 'amb')):
            ambient_oc_df.loc[row['chamber_block'] == ambient_oc_df['chamber_block'],'corresponding_chamber_block'] = current_oc_ch4_block
            ch4_oc_amb_found = True
        if((ch16_oc_amb_found == False) & (row['status'] == 'amb')):
            ambient_oc_df.loc[row['chamber_block'] == ambient_oc_df['chamber_block'],'corresponding_chamber_block'] = current_oc_ch16_block
            ch16_oc_amb_found = True
    # Combine the ambient dfs
    ambient_df = ambient_df.append(ambient_oc_df)
    ambient_df.sort_values(["chamber_block"], inplace=True)
    # Remove some of the unnecessary data
    oc_df = oc_df.loc[oc_df['corresponding_chamber_block'].notna()]
    oc_df['corresponding_chamber_block'] = oc_df['corresponding_chamber_block'].astype(int)
    oc_df.drop(['chamber','status','chamber_block'], axis=1, inplace=True)
    ambient_df = ambient_df.loc[ambient_df['corresponding_chamber_block'].notna()]
    ambient_df['corresponding_chamber_block'] = ambient_df['corresponding_chamber_block'].astype(int)
    ambient_df.drop(['chamber','status','chamber_block'], axis=1, inplace=True)
    
    # Rename columns: Add suffixes as identifiers
    oc_df = oc_df.add_suffix('.oc')
    oc_df.rename(columns={'corresponding_chamber_block.oc':'chamber_block'}, inplace=True)
    ambient_df = ambient_df.add_suffix('.ambient')
    ambient_df.rename(columns={'corresponding_chamber_block.ambient':'chamber_block'}, inplace=True)
    
    # Merge with data
    merged = temp.merge(ambient_df, on="chamber_block", how="left")
    merged = merged.merge(oc_df, on="chamber_block", how="left")
    
    # Remove unused data
    merged = merged.loc[merged['chamber'].notna()]
    
    return(merged)

# Calculate all fluxes for all gases
def calculate_fluxes(temp):
    temp = temp.copy()
    
    # The TSI flow meters use SLPM with the following standard values for air flow:
    air_temperature_tsi_c = 21.11 # [°C]
    air_pressure_tsi_kpa  = 101.3 # [kPa]
    
    # Fluxes with total leaf area, between chamber and ambient measurements
    temp['flux.h2o.ch_amb.mmol_m2_s'] = 10**3 * calculate_h2o_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.ambient'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2'])
    temp['flux.co2.ch_amb.umol_m2_s'] = 10**6 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.ambient'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['co2.mean.umol_mol.ambient'] * 10**(-6),
                                                            temp['co2.mean.umol_mol'] * 10**(-6),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2'])
    temp['flux.cos.ch_amb.pmol_m2_s'] = 10**12 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.ambient'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['cos.mean.pmol_mol.ambient'] * 10**(-12),
                                                            temp['cos.mean.pmol_mol'] * 10**(-12),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2'])
    temp['flux.co.ch_amb.nmol_m2_s'] = 10**9 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.ambient'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['co.nmol_mol.ambient'] * 10**(-9),
                                                            temp['co.nmol_mol'] * 10**(-9),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2'])
    
    # Fluxes with projected leaf area, between chamber and ambient measurements
    temp['flux.h2o.ch_amb.proj_la.mmol_m2_s'] = 10**3 * calculate_h2o_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.ambient'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2']/proj_la_factor)
    temp['flux.co2.ch_amb.proj_la.umol_m2_s'] = 10**6 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.ambient'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['co2.mean.umol_mol.ambient'] * 10**(-6),
                                                            temp['co2.mean.umol_mol'] * 10**(-6),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2']/proj_la_factor)
    temp['flux.cos.ch_amb.proj_la.pmol_m2_s'] = 10**12 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.ambient'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['cos.mean.pmol_mol.ambient'] * 10**(-12),
                                                            temp['cos.mean.pmol_mol'] * 10**(-12),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2']/proj_la_factor)
    temp['flux.co.ch_amb.proj_la.nmol_m2_s'] = 10**9 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.ambient'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['co.nmol_mol.ambient'] * 10**(-9),
                                                            temp['co.nmol_mol'] * 10**(-9),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2']/proj_la_factor)
    
    # Fluxes with total leaf area, between chamber and open chamber measurements
    temp['flux.h2o.ch_oc.mmol_m2_s'] = 10**3 * calculate_h2o_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.oc'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2'])
    temp['flux.co2.ch_oc.umol_m2_s'] = 10**6 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.oc'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['co2.mean.umol_mol.oc'] * 10**(-6),
                                                            temp['co2.mean.umol_mol'] * 10**(-6),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2'])
    temp['flux.cos.ch_oc.pmol_m2_s'] = 10**12 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.oc'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['cos.mean.pmol_mol.oc'] * 10**(-12),
                                                            temp['cos.mean.pmol_mol'] * 10**(-12),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2'])
    temp['flux.co.ch_oc.nmol_m2_s'] = 10**9 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.oc'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['co.nmol_mol.oc'] * 10**(-9),
                                                            temp['co.nmol_mol'] * 10**(-9),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2'])
    
    # Fluxes with projected leaf area, between chamber and open chamber measurements
    temp['flux.h2o.ch_oc.proj_la.mmol_m2_s'] = 10**3 * calculate_h2o_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.oc'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2']/proj_la_factor)
    temp['flux.co2.ch_oc.proj_la.umol_m2_s'] = 10**6 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.oc'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['co2.mean.umol_mol.oc'] * 10**(-6),
                                                            temp['co2.mean.umol_mol'] * 10**(-6),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2']/proj_la_factor)
    temp['flux.cos.ch_oc.proj_la.pmol_m2_s'] = 10**12 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.oc'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['cos.mean.pmol_mol.oc'] * 10**(-12),
                                                            temp['cos.mean.pmol_mol'] * 10**(-12),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2']/proj_la_factor)
    temp['flux.co.ch_oc.proj_la.nmol_m2_s'] = 10**9 * calculate_gas_flux(air_temperature_tsi_c,
                                                            air_pressure_tsi_kpa*1000,
                                                            temp['h2o.mmol_mol.oc'],
                                                            temp['h2o.mmol_mol'],
                                                            temp['co.nmol_mol.oc'] * 10**(-9),
                                                            temp['co.nmol_mol'] * 10**(-9),
                                                            temp['pump.flow.current.chamber.slpm'],
                                                            temp['leaf.area.m2']/proj_la_factor)
    temp.loc[temp['status'] == 'oc', 'flux.h2o.ch_oc.mmol_m2_s'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.co2.ch_oc.umol_m2_s'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.cos.ch_oc.pmol_m2_s'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.co.ch_oc.nmol_m2_s'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.h2o.ch_oc.proj_la.mmol_m2_s'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.co2.ch_oc.proj_la.umol_m2_s'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.cos.ch_oc.proj_la.pmol_m2_s'] = np.nan
    temp.loc[temp['status'] == 'oc', 'flux.co.ch_oc.proj_la.nmol_m2_s'] = np.nan
    
    return(temp)

def calibrate_co2_h2o(temp, cal_stats):
    temp = temp.copy()
    
    # Correlation of H2O
    print('    Correlation between IRGA ambient and Laser')
    h2o_mask = (temp['status'] == 'amb') & (temp['h2o.irga.ambient.mmol_mol'].notna()) & (temp['h2o.mmol_mol'].notna())
    n_h2o = len(temp.loc[h2o_mask,'h2o.mmol_mol'])
    if(n_h2o >= 10):
        h2o_corr = stats.linregress(temp.loc[h2o_mask,'h2o.mmol_mol'], temp.loc[h2o_mask,'h2o.irga.ambient.mmol_mol'])
        print('        H2O: ', h2o_corr, n_h2o)
    # Correlation of CO2
    co2_mask = (temp['status'] == 'amb') & (temp['co2.irga.ambient.umol_mol'].notna()) & (temp['co2.mean.umol_mol'].notna())
    n_co2 = len(temp.loc[co2_mask,'co2.mean.umol_mol'])
    if(n_co2 >= 10):
        co2_corr = stats.linregress(temp.loc[co2_mask,'co2.mean.umol_mol'], temp.loc[co2_mask,'co2.irga.ambient.umol_mol'])
        print('        CO2: ', co2_corr, n_co2)
    
    # Flag the data:
    # H2O bad  = 1
    # CO2 bad  = 2
    # Both bad = 3
    temp['flag'] = np.nan
    if((n_h2o < 10) | (n_co2 < 10)):
        temp['flag'] = 0
    else:
        if(h2o_corr.rvalue < 0.90):
            temp['flag'] = 1
        if(co2_corr.rvalue < 0.90):
            temp['flag'] = 2
        if((h2o_corr.rvalue < 0.90) & (co2_corr.rvalue < 0.90)):
            temp['flag'] = 3
        
    # Add information to the calibration statistics df
    temp_cal_df = pd.DataFrame(columns=['date','h2o.slope','h2o.intercept','h2o.r2','h2o.p','h2o.sd','h2o.n','co2.slope','co2.intercept','co2.r2','co2.p','co2.sd','co2.n'], index=[0])
    temp_cal_df['date'] = temp['timestamp'].dt.date.unique()
    if(n_h2o >= 10):
        temp_cal_df['h2o.slope']     = h2o_corr.slope
        temp_cal_df['h2o.intercept'] = h2o_corr.intercept
        temp_cal_df['h2o.r2']        = h2o_corr.rvalue
        temp_cal_df['h2o.p']         = h2o_corr.pvalue
        temp_cal_df['h2o.sd']        = h2o_corr.stderr
    temp_cal_df['h2o.n'] = n_h2o
    if(n_co2 >= 10):
        temp_cal_df['co2.slope']     = co2_corr.slope
        temp_cal_df['co2.intercept'] = co2_corr.intercept
        temp_cal_df['co2.r2']        = co2_corr.rvalue
        temp_cal_df['co2.p']         = co2_corr.pvalue
        temp_cal_df['co2.sd']        = co2_corr.stderr
    temp_cal_df['co2.n'] = n_co2
    cal_stats = cal_stats.append(temp_cal_df)
        
    # Apply corrections
    if(n_h2o >= 10):
        if(h2o_corr.rvalue > 0.75):
            temp['h2o.calibrated.mmol_mol'] = h2o_corr.slope * temp['h2o.mmol_mol'] + h2o_corr.intercept
    if(n_co2 >= 10):
        if(co2_corr.rvalue > 0.50):
            temp['co2.calibrated.umol_mol'] = co2_corr.slope * temp['co2.mean.umol_mol'] + co2_corr.intercept
    
    return(temp, cal_stats)

# Writes output files averaged by chamber and time
def write_stats_file(out_df, out_dir):
    # Check if output folders exist. If not, create
    if(not os.path.exists(out_dir)):
        os.makedirs(out_dir)
    # Create file name
    out_fn = out_dir + '/calibration_stats_co2_h2o.csv'
    # Remove timezone information
    #out_df['timestamp'] = out_df['timestamp'].dt.tz_localize(None)
    # If the file does not exist, create it with the header
    out_df.to_csv(out_fn, index=False, encoding='utf-8', date_format='%Y-%m-%d')

# Writes output files averaged by chamber and time
def write_summarised_file(out_df, date_idx, out_dir):
    # Check if output folders exist. If not, create
    this_month = str(date_idx)[0:6]
    if(not os.path.exists(out_dir)): # make directory if it doesn't exist
        os.makedirs(out_dir)
    # Create file name
    out_fn = out_dir + '/' + output_fn + '_' + str(this_month) + '.csv'
    # Remove timezone information
    #out_df['timestamp'] = out_df['timestamp'].dt.tz_localize(None)
    # If the monthly file does not exist, create it with the header
    if not os.path.isfile(out_fn):
        out_df.to_csv(out_fn, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S')
    else: # otherwise append without writing the header
        out_df.to_csv(out_fn, mode='a', header=False, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S')

# Full processing pipeline

In [ ]:

# Applies all the corrections in order. This gets called by the main loops that are in control of reading and writing files
def apply_corrections(laser, amb, par, tc, flow, cal_stats, date_idx):
    # Show info
    print("- Applying corrections on ", np.unique(laser['dayid'])[0], "...")
    # Add average columns of the different gases, and standard deviation
    laser = make_gas_means(laser)
    # get the right PAR data
    par_df = get_PAR(par, date_idx)
    # Merge all data # TODO: deal with not having one of them
    temp = add_ambient(laser, amb)
    temp = add_PAR(temp, par_df)
    temp = add_temperature(temp, tc_df)
    temp = add_pump_flow(temp, flow_df)
    # Temporarily add counter and chamber
    temp = make_counter_and_chamber(temp, date_idx)
    # Checks if there is enough data left to properly do all the calculations
    if(len(temp['timestamp']) > 240):
        # Add a column for current PAR, temperature and current flow
        temp = add_current_variable(temp)
        # Add leaf area
        temp = add_leaf_area(temp, date_idx)
        # Add the pump flow rate from the log file, if not previously given by measurements
        temp = add_log_flow(temp)
        # Remove bad chambers
        temp = remove_bad_chambers(temp)
        # Remove the data outside the averaging period, if specified
        temp = remove_obsolete_data(temp, averaging_period)
        # Daily correction of CO2 and H2O by correlating to IRGA values, if available
        temp, cal_stats = calibrate_co2_h2o(temp, cal_stats)
    else:
        print("- Data of", date_idx, "too short for corrections")
        
    return(temp, cal_stats)

def calculate_values(temp, date_idx):
    # Make means, standard deviations, and
    # remove all except last number of seconds, provided in the variable 'averaging_period'
    temp = make_means(temp, averaging_period)
    # Find the corresponding ambient measurements
    if(find_ambient == 'interpolated'):
        temp = find_corresponding_ambient_interpolated(temp)
    elif(find_ambient == 'next'):
        temp = find_corresponding_ambient_next(temp)
    else:
        raise ValueError('Invalid choice: Please choose which method is used to find ambient data')
    # calculate fluxes
    temp = calculate_fluxes(temp)
    
    return(temp)

In [ ]:
# Run calculations
#-----------------

# List all files in the directories
laser_fn_list = sorted(glob.glob(project_path_laser + '*/*', recursive=True))
amb_fn_list   = sorted(glob.glob(project_path_amb + '*/*', recursive=True))
par_fn_list   = sorted(glob.glob(project_path_par + '*/*', recursive=True))
tc_fn_list    = sorted(glob.glob(project_path_tc + '*/*', recursive=True))
flow_fn_list  = sorted(glob.glob(project_path_flow + '*/*', recursive=True))
saved = []

# Prepare 1h directory. Because files aren't overwritten typically, we need to empty it first
empty_dir(project_path_output + "1h/")
empty_dir(project_path_output + "1min/")
empty_dir(project_path_output + 'stats_calibration/')

# Prepare empty dataframe for calibration statistics
cal_stats_df = pd.DataFrame(columns=['date','h2o.slope','h2o.intercept','h2o.r2','h2o.p','h2o.sd','h2o.n','co2.slope','co2.intercept','co2.r2','co2.p','co2.sd','co2.n'])

for fn_i, fn in enumerate(laser_fn_list):
    # Only run data in the month list
    current_month = fn.replace(project_path_laser[:-1], "")[1:8]
    if((current_month in month_list) or (len(month_list) == 0)):
        pass
    else:
        continue
    
    # Determine current day
    this_day = fn[-12:-4]
    
    # Debugging message
    if(fn_i % 1 == 0): # % 20 to show every 20th file being loaded
        print( '{:<07}'.format(str(round(fn_i * 100 / len(laser_fn_list), 4))) + "%\t" + str(this_day[0:4]) + '-' + str(this_day[4:6]) + '-' + str(this_day[6:8])) # Show status
        
    # Now find corresponding files and load all the data
    this_day = fn[-12:-4]
    laser_df = read_data_file(fn)
    amb_df  = find_and_load_file(amb_fn_list, this_day)
    par_df  = find_and_load_file(par_fn_list, this_day)
    tc_df   = find_and_load_file(tc_fn_list, this_day)
    flow_df = find_and_load_file(flow_fn_list, this_day)
    
    # Now apply corrections
    out_df, cal_stats_df = apply_corrections(laser_df, amb_df, par_df, tc_df, flow_df, cal_stats_df, this_day)
    
    # If there is any data left
    if(len(out_df) > 0):
        # Write output in full temporal resolution
        write_output_file(out_df, this_day, project_path_output + 'raw_qc/')
        
        # Create means and calculate fluxes
        out_df = calculate_values(out_df, this_day)
        
        # Write output averaged output file
        write_summarised_file(out_df, this_day, project_path_output + '1min/')
    pass

write_stats_file(cal_stats_df, project_path_output + 'stats_calibration/')
print("Done...")